In [17]:
import sys, os, shutil
import gc
import tempfile
import warnings

from glob import glob
from time import time
import numpy as np

import pandas as pd
from dask import dataframe as dd

## Construção da base de dados

A seguir, é executado o processo de obtenção da base de dados final utilizada na análise dos dados do Tribunal de Contas. Temos acesso à planilhas anuais com todos os gastos declarados pelos municípios, os quais foram concedidos pelo Tribunal de Contas. Neste caso temos o interesse apenas naquelas despesas cujo tipo seja ''Valor Liquidado'', que representam aqueles valores de fato concedidos pelo governo do estado.

A seguir construímos a tabela correspondente aos dados de 2010 manualmente, que servirá de base para a criação de uma função para a automatização deste processo de construção das bases de dados.

### Biblioteca dask

Devido à imensa quantidade de dados contida nas tabelas de despesas anuais, não é desejável o carregamento destas tabelas diretamente pela biblitoeca pandas, que armazena toda a informação na memória RAM do computador. Considerando um computador com 16GB de memória, não é possível o tratamento dos dados brutos desta maneira com facilidade. Por essa razão utilizamos a biblioteca paralelizada dask, que tem a capacidade de carregar os conjuntos de dados de modo a armazenar em memória apenas as informações de interesse a serem tratadas.

In [2]:
df_dd = dd.read_csv("despesas-2010.csv", sep = ";", encoding = "ISO-8859-1", assume_missing=True, decimal = ",")
display( df_dd.head(3) )

,id_despesa_detalhe,ano_exercicio,ds_municipio,ds_orgao,mes_referencia,mes_ref_extenso,tp_despesa,nr_empenho,identificador_despesa,ds_despesa,...,ds_subfuncao_governo,cd_programa,ds_programa,cd_acao,ds_acao,ds_fonte_recurso,ds_cd_aplicacao_fixo,ds_modalidade_lic,ds_elemento,historico_despesa
0,49141808.0,2010.0,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,1.0,janeiro,Empenhado,707-2010,CNPJ - PESSOA JURÍDICA - 61684387000137,FREAUTO PEÇAS E SERVIÇOS LTDA.,...,ENSINO FUNDAMENTAL,15.0,PROG.DE DESENVOLVIMENTO EDUCACIONAL,2029.0,GESTAO DOS RECURSOS QESE,TRANSFERÊNCIAS E CONVÊNIOS FEDERAIS-VINCULADOS,0200 - EDUCAÇÃO - RECURSOS ESPECÍFICOS,DISPENSA DE LICITAÇÃO,33903039 - MATERIAL PARA MANUTENÇÃO DE VEÍCULOS,Valor empenhado conforme Pedido de Compras 282...
1,49138019.0,2010.0,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,1.0,janeiro,Empenhado,694-2010,PESSOA FÍSICA - 949588,DOUGLAS LUIZ MONTELO,...,ENSINO FUNDAMENTAL,15.0,PROG.DE DESENVOLVIMENTO EDUCACIONAL,2029.0,GESTAO DOS RECURSOS QESE,TRANSFERÊNCIAS E CONVÊNIOS FEDERAIS-VINCULADOS,0200 - EDUCAÇÃO - RECURSOS ESPECÍFICOS,DISPENSA DE LICITAÇÃO,33903620 - MANUTENÇÃO E CONSERVAÇÃO DE VEÍCULOS,Valor empenhado conforme Pedido de Compras 269...
2,49133666.0,2010.0,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,1.0,janeiro,Empenhado,79-2010,CNPJ - PESSOA JURÍDICA - 07753097000152,KAENE CONSTRUTORA LTDA,...,EDUCAÇÃO INFANTIL,15.0,PROG.DE DESENVOLVIMENTO EDUCACIONAL,1011.0,CONSTR/AMPL/REFORMA DE CRECHES,TESOURO,0200 - EDUCAÇÃO - RECURSOS ESPECÍFICOS,TOMADA DE PREÇOS,44905191 - OBRAS EM ANDAMENTO,Valor empenhado conforme Pedido de Compras 14/...


Acima ao chamarmos o método ''head'', os dados foram recuperados do arquivo dinamicamente. Como podemos ver a seguir, o objeto consiste em uma ''máscara'' dos dados originais.

In [3]:
display( df_dd )

,id_despesa_detalhe,ano_exercicio,ds_municipio,ds_orgao,mes_referencia,mes_ref_extenso,tp_despesa,nr_empenho,identificador_despesa,ds_despesa,dt_emissao_despesa,vl_despesa,ds_funcao_governo,ds_subfuncao_governo,cd_programa,ds_programa,cd_acao,ds_acao,ds_fonte_recurso,ds_cd_aplicacao_fixo,ds_modalidade_lic,ds_elemento,historico_despesa
npartitions=162,,,,,,,,,,,,,,,,,,,,,,,
,float64,float64,object,object,float64,object,object,object,object,object,object,float64,object,object,float64,object,float64,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


Com essa funcionalidade em mente, simplificamos todo o processo de sumarização desta base fazendo o uso do método loc para selecionar apenas as despesas de interesse para a criação da base (''Valor Liquidado''). Em seguida, os dados já filtrados são salvos em 8 arquivos de extensão parquet, os quais podem ser tranquilamente carregados e processados pela biblioteca pandas para processamento.

In [4]:
warnings.filterwarnings("ignore")

# Filtra as informações
df_dd = df_dd.loc[df_dd.tp_despesa == "Valor Liquidado", :]
# Define o número de partições do dataframe e salva em formato .parquet
df_dd = df_dd.repartition(npartitions = 8)
dd.to_parquet(df_dd, "2010", write_index = False)

Uma vez salvo o dataframe em diferentes arquivos, basta percorrermos todos eles, já agregando as despesas totais segundo as datas e os códigos de municípios, órgãos e despesas.

In [7]:
arquivos = glob('2010/*.parquet')
print(arquivos)

['2010/part.4.parquet', '2010/part.5.parquet', '2010/part.3.parquet', '2010/part.2.parquet', '2010/part.0.parquet', '2010/part.6.parquet', '2010/part.7.parquet', '2010/part.1.parquet']


In [6]:
df = None
for arquivo in arquivos:
    df_aux = pd.read_parquet(arquivo)
    # Agrega as despesas segundo as datas e os códigos indicadores de município, órgão e despesa
    df_aux = (
        df_aux.
        groupby(["ano_exercicio", "mes_referencia", "ds_municipio", "ds_orgao", "ds_elemento"]).
        agg({
            "vl_despesa": "sum",
        }).
        reset_index()
    )
    if(df is None):
        df = df_aux.copy()
    else:
        df = pd.concat([df, df_aux], axis = 0)
# Agrega todos os valores novamente, garantindo que despesas iguais em diferentes arquivos sejam consideradas como iguais        
df = (
    df.
    groupby(["ano_exercicio", "mes_referencia", "ds_municipio", "ds_orgao", "ds_elemento"]).
    agg({
        "vl_despesa": "sum",
    }).
    reset_index()
)
print("Tamanho final da tabela: {}".format(df.shape))
display( df.head(3) )

Tamanho final da tabela: (750715, 6)


,ano_exercicio,mes_referencia,ds_municipio,ds_orgao,ds_elemento,vl_despesa
0,2010.0,1.0,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31900187 - COMPLEMENTAÇÃO DE APOSENTADORIAS,15550.64
1,2010.0,1.0,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31900386 - COMPLEMENTAÇÃO DE PENSÕES - PESSOAL...,1428.79
2,2010.0,1.0,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31900415 - OBRIGAÇÕES PATRONAIS,28524.52


## Correspondência códigos de despesa

In [52]:
codigos = pd.DataFrame({"ds_elemento": df.ds_elemento})
codigos[['cod_subitem', 'ds_subitem']] = codigos.ds_elemento.str.split(pat=' - ', n = 1, expand=True)
codigos['ds_subitem'] = codigos['ds_subitem'].str.strip()

codigos = codigos.filter(["cod_subitem", "ds_subitem"])
codigos.head(3)

,cod_subitem,ds_subitem
0,31900187,COMPLEMENTAÇÃO DE APOSENTADORIAS
1,31900386,COMPLEMENTAÇÃO DE PENSÕES - PESSOAL CIVIL
2,31900415,OBRIGAÇÕES PATRONAIS


In [53]:
codigos.loc[:,"cod_subitem"] = codigos.cod_subitem.apply(lambda x : x[-4:])
codigos = codigos.drop_duplicates()

codigos.head(3)

,cod_subitem,ds_subitem
0,0187,COMPLEMENTAÇÃO DE APOSENTADORIAS
1,0386,COMPLEMENTAÇÃO DE PENSÕES - PESSOAL CIVIL
2,0415,OBRIGAÇÕES PATRONAIS


In [58]:
codigos.to_excel("classificacoes_despesas.xlsx")

In [57]:
codigos.loc[codigos.cod_subitem == '3001',:]

,cod_subitem,ds_subitem
48,3001,COMBUSTÍVEIS E LUBRIFICANTES AUTOMOTIVOS
35004,3001,COMBUSTÍVEIS E LUBRIFICANTES AUTOMOTIVOS - INT...


## Generalização em uma função

Para formatarmos toda a base de dados com diferentes valores anuais, façamos uma função que carrega uma única base de dados e unifica todas as informações da mesma forma abordada acima para o ano de 2010. Vale destacar que, uma vez que utilizaremos uma grande quantidade de dados, o código cria as bases necessárias e em seguida as deleta do disco, resultando apenas na tabela final já processada.

In [2]:
def criar_tabela_ano(data_path, out_path = None, show_temp_dir = False):
    data_name = data_path[:-4]
    df_dd = dd.read_csv(data_path, sep = ";", encoding = "ISO-8859-1", assume_missing = True, decimal = ",", dtype = {"nr_identificador_despesa": "object"})
    
    # Cria um diretório temporário para o arquivo segmentado (parquet)
    temp_dir = tempfile.TemporaryDirectory().name
    if(show_temp_dir):
        print("Armazenando arquivos auxiliares em {}".format(temp_dir))
    
    warnings.filterwarnings("ignore")
    # Separa o arquivo grande em 8 partes menores
    df_dd = df_dd.loc[df_dd.tp_despesa == "Valor Liquidado", ["ano_exercicio", "mes_referencia", "ds_municipio", "ds_orgao", "ds_elemento", "vl_despesa"]]
    df_dd = df_dd.repartition(npartitions = 8)
    dd.to_parquet(df_dd, "{}/{}".format(temp_dir, data_name), write_index = False)    

    # Percorre todos os arquivos segmentados, sumariznando e contatenando a uma única tabela
    arquivos = glob("{}/{}/*.parquet".format(temp_dir, data_name))
    df = None
    for arquivo in arquivos:
        df_aux = pd.read_parquet(arquivo)
        df_aux = (
            df_aux.
            groupby(["ano_exercicio", "mes_referencia", "ds_municipio", "ds_orgao", "ds_elemento"]).
            agg({
                "vl_despesa": "sum",
            }).
            reset_index()
        )
        if(df is None):
            df = df_aux.copy()
        else:
            df = pd.concat([df, df_aux], axis = 0)        
            
    # Agrega todos os valores novamente, garantindo que despesas iguais em diferentes arquivos sejam consideradas como iguais        
    df = (
        df.
        groupby(["ano_exercicio", "mes_referencia", "ds_municipio", "ds_orgao", "ds_elemento"]).
        agg({
            "vl_despesa": "sum",
        }).
        reset_index()
    )
    
    # Uma vez obtida a tabela já agregada, deleta as pastas auxiliares do disco
    shutil.rmtree(temp_dir)
    
    if(out_path is None):
        return df
    
    df.to_parquet(out_path)

In [3]:
criar_tabela_ano("despesas-2010.csv", out_path = "Data/2010.parquet")

In [6]:
criar_tabela_ano("despesas-2011.csv", out_path = "Data/2011.parquet")

In [4]:
criar_tabela_ano("despesas-2012.csv", out_path = "Data/2012.parquet")

In [6]:
criar_tabela_ano("despesas-2013.csv", out_path = "Data/2013.parquet")

In [29]:
criar_tabela_ano("despesas-2014.csv", out_path = "Data/2014.parquet")

In [31]:
criar_tabela_ano("despesas-2015.csv", out_path = "Data/2015.parquet")

In [34]:
criar_tabela_ano("despesas-2016.csv", out_path = "Data/2016.parquet")

In [6]:
criar_tabela_ano("despesas-2017.csv", out_path = "Data/2017.parquet")

In [8]:
criar_tabela_ano("despesas-2018.csv", out_path = "Data/2018.parquet")

In [ ]:
criar_tabela_ano("despesas-2019.csv", out_path = "Data/2019.parquet")

In [4]:
criar_tabela_ano("despesas-2020.csv", out_path = "Data/2020.parquet")

In [14]:
criar_tabela_ano("despesas-2021.csv", out_path = "Data/2021.parquet")

In [16]:
criar_tabela_ano("despesas-2022.csv", out_path = "Data/2022.parquet")

Os dados de origem .csv foram obtidos em https://transparencia.tce.sp.gov.br/conjunto-de-dados. Os dados não estão disponibilizados junto aos arquivos devido ao seu excessivo tamanho. Apenas os valores já agrupados segundo as despesas estão disponíveis.

Uma vez agrupados e separados em cada ano, basta reunirmos todas as informações em uma única tabela.

In [2]:
arquivos = ["Data/{}.parquet".format(i) for i in range(2010, 2022+1)]
print( arquivos )

['Data/2010.parquet', 'Data/2011.parquet', 'Data/2012.parquet', 'Data/2013.parquet', 'Data/2014.parquet', 'Data/2015.parquet', 'Data/2016.parquet', 'Data/2017.parquet', 'Data/2018.parquet', 'Data/2019.parquet', 'Data/2020.parquet', 'Data/2021.parquet', 'Data/2022.parquet']


In [3]:
df = None
for arquivo in arquivos:
    df_aux = pd.read_parquet(arquivo)
    if(df is None):
        df = df_aux.copy()
    else:
        df = pd.concat([df, df_aux], axis = 0)
        
print( df.dtypes )
display( df.head(3) )

ano_exercicio     float64
mes_referencia    float64
ds_municipio       object
ds_orgao           object
ds_elemento        object
vl_despesa        float64
dtype: object


,ano_exercicio,mes_referencia,ds_municipio,ds_orgao,ds_elemento,vl_despesa
0,2010.0,1.0,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31900187 - COMPLEMENTAÇÃO DE APOSENTADORIAS,15550.64
1,2010.0,1.0,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31900386 - COMPLEMENTAÇÃO DE PENSÕES - PESSOAL...,1428.79
2,2010.0,1.0,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31900415 - OBRIGAÇÕES PATRONAIS,28524.52


In [4]:
df.ano_exercicio = df.ano_exercicio.astype(int)
df.mes_referencia = df.mes_referencia.astype(int)
df = df.reset_index(drop = True)

df[['cod_subitem', 'ds_subitem']] = df.ds_elemento.str.split(pat=' - ', n = 1, expand=True)
df['ds_subitem'] = df['ds_subitem'].str.strip()
display( df.head(3) )

df.to_parquet("Data/data2010to2022.parquet")

,ano_exercicio,mes_referencia,ds_municipio,ds_orgao,ds_elemento,vl_despesa,cod_subitem,ds_subitem
0,2010,1,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31900187 - COMPLEMENTAÇÃO DE APOSENTADORIAS,15550.64,31900187,COMPLEMENTAÇÃO DE APOSENTADORIAS
1,2010,1,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31900386 - COMPLEMENTAÇÃO DE PENSÕES - PESSOAL...,1428.79,31900386,COMPLEMENTAÇÃO DE PENSÕES - PESSOAL CIVIL
2,2010,1,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31900415 - OBRIGAÇÕES PATRONAIS,28524.52,31900415,OBRIGAÇÕES PATRONAIS


Finalmente, antes de tratarmos da análise destes dados devemos fazer mais algumas correções, sugeridas como forma de auxiliar o ajuste dos modelos. Essa correção consiste na refatoração de despesas pouco comuns ou financeiramente ''irrelevantes'', no sentido de que não impactam em grande escala as despesas do estado. Algumas despesas serão retiradas do conjunto de dados, enquanto outras serão agrupadas entre si.

In [5]:
# df = pd.read_parquet("Data/data2010to2022.parquet")

In [8]:
# Índices de despesas que devem ser agrupadas em apenas uma
agrupar = ['01','03','05','13','14','16','21','31','32','33','34','35','40','43','46','47','48','61','62','65','70',
           '71','73','76','83','91','92','93','94']
# Índices de despesas que devem ser ignoradas
ignorar = ['04','08','07','09','10','18','19','20','22','23','24','25','27','37','38','41','42','45','49','59', '62', '64', 
           '65', '66','67','72','74','75', '76','77', '81','82','83','84','86','95','96','97','98','99'] #modalidade 50 fica fora do modelo

dividir = ['11','30','36','39','51','52']
ti = ['3908', '3911', '3957', '3994', '3995', '3997']

substituir = {'0100':'0101','0300':'0301', '1100':'1101', '1187':'1101', '3000':'3099', '3600':'3699', '3900':'3999', '5100':'5199', '5200':'5299'}

In [9]:
df[['cod_subitem', 'ds_subitem']] = df.ds_elemento.str.split(pat=' - ', n=1, expand=True)
df['ds_subitem'] = df['ds_subitem'].str.strip()

# # Separação do código de despesa segundo seus diferentes setores
df['grupo'] = df.cod_subitem.str[:2]
df['modalidade'] = df.cod_subitem.str[2:4]
df['elemento'] = df.cod_subitem.str[4:6]
df['subitem'] = df.cod_subitem.str[6:8]

In [10]:
# Exclusão de terceiro setor e índices pouco representativos
print(f'Quantidade de linhas total: {len(df.index)}')
print(f'Quantidade de linhas a serem excluídas: {len(df.loc[df.elemento.isin(ignorar)])}')
print(f'Quantidade de linhas esperada após exclusão: {len(df.index)-len(df.loc[df.elemento.isin(ignorar)])}')
df = df.loc[~df.elemento.isin(ignorar)]

print(f'Quantidade de linhas após a exclusão: {len(df.index)}')
print(f'Quantidade de linhas terceiro setor: {len(df.loc[(df.modalidade=="50")|(df.cod_subitem.str[4:8]=="3975")].index)}')
print(f'Quantidade de linhas esperada após exclusão: {len(df.index)-len(df.loc[(df.modalidade=="50")|(df.cod_subitem.str[4:8]=="3975")].index)}')
df = df.loc[(df.modalidade!='50')&(df.cod_subitem.str[4:8]!='3975')&(df.cod_subitem.str[4:8]!='8500')]

print(f'Quantidade de linhas após exclusão do terceiro setor: {len(df.index)}')

df = df.reset_index(drop = True)
df.to_parquet("Data/data2010to2022.parquet")

Quantidade de linhas total: 10566958
Quantidade de linhas a serem excluídas: 240562
Quantidade de linhas esperada após exclusão: 10326396
Quantidade de linhas após a exclusão: 10326396
Quantidade de linhas terceiro setor: 104426
Quantidade de linhas esperada após exclusão: 10221970
Quantidade de linhas após exclusão do terceiro setor: 10221970


In [11]:
# Agrupamento
df.loc[df.cod_subitem.str[4:8].isin(ti), 'id_despesa'] = '4000'
df.loc[df.elemento.isin(agrupar), 'id_despesa'] = df.elemento+'00'
df.loc[df.elemento.isin(dividir), 'id_despesa'] = df.elemento+df.subitem
df.id_despesa.replace(substituir, inplace=True)

In [12]:
df = (
    df.
    groupby(['ano_exercicio', 'mes_referencia', 'ds_municipio', 'ds_orgao', 'id_despesa']).
    agg({
        "vl_despesa": "sum"
    }).
    reset_index()
)

print("Tamanho da tabela: {}".format(df.shape))

df.to_parquet("Data/data2010to2022.parquet")

Tamanho da tabela: (9529123, 6)


## Comparação com a base utilizada originalmente

Façamos uma breve comparação entre a base de dados utilizada anteriormente e essa nova base gerada.

In [2]:
df = pd.read_parquet("Data/data2010to2022.parquet")
df2 = pd.read_parquet("Data/base2022subitensagrupadospormesV2.parquet")

print("Dimensões: {} e {}".format(df.shape, df2.shape))

display( df.head(2) )
display( df2.head(2) )

Dimensões: (9529123, 6) e (9509919, 9)


,ano_exercicio,mes_referencia,ds_municipio,ds_orgao,id_despesa,vl_despesa
0,2010,1,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,0101,15550.64
1,2010,1,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,0301,1428.79


,ano_exercicio,mes_referencia,cd_municipio_ibge,entidade_id,ds_municipio,ds_orgao,ds_tp_entidade,id_despesa,vl_despesa
0,2010,1,3500105.0,16799.0,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,PREFEITURA MUNICIPAL,0101,208321.50
1,2010,1,3500105.0,16799.0,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,PREFEITURA MUNICIPAL,0301,84729.15


In [27]:
# Rápida correção dos nomes de alguns municípios, não padronizados na base do portal da transparência

# Municípios
df.loc[df.ds_municipio == "Biritiba-Mirim", "ds_municipio"] = "Biritiba Mirim"
df.loc[df.ds_municipio == "Florínea", "ds_municipio"] = "Florínia"
df.loc[df.ds_municipio == "Itaóca", "ds_municipio"] = "Itaoca"
df.loc[df.ds_municipio == "São Luis do Paraitinga", "ds_municipio"] = "São Luíz do Paraitinga"
df.loc[df.ds_municipio == "São Luís do Paraitinga", "ds_municipio"] = "São Luíz do Paraitinga"
df.loc[df.ds_municipio == "São Luíz do Paraitinga", "ds_municipio"] = "São Luiz do Paraitinga"

# Órgãos
df.loc[df.ds_orgao == "CÂMARA MUNICIPAL DE BIRITIBA-MIRIM", "ds_orgao"] = "CÂMARA MUNICIPAL DE BIRITIBA MIRIM"
df.loc[df.ds_orgao == "PREFEITURA MUNICIPAL DE BIRITIBA-MIRIM", "ds_orgao"] = "PREFEITURA MUNICIPAL DE BIRITIBA MIRIM"
df.loc[df.ds_orgao == "PREFEITURA MUNICIPAL DE FLORÍNEA", "ds_orgao"] = "PREFEITURA MUNICIPAL DE FLORÍNIA"
df.loc[df.ds_orgao == "CÂMARA MUNICIPAL DE FLORÍNEA", "ds_orgao"] = "CÂMARA MUNICIPAL DE FLORÍNIA"
df.loc[df.ds_orgao == "CÂMARA MUNICIPAL DE ITAÓCA", "ds_orgao"] = "CÂMARA MUNICIPAL DE ITAOCA"
df.loc[df.ds_orgao == "PREFEITURA MUNICIPAL DE ITAÓCA", "ds_orgao"] = "PREFEITURA MUNICIPAL DE ITAOCA"
df.loc[df.ds_orgao == "CÂMARA MUNICIPAL DE SÃO LUÍS DO PARAITINGA", "ds_orgao"] = "CÂMARA MUNICIPAL DE SÃO LUÍZ DO PARAITINGA"
df.loc[df.ds_orgao == "PREFEITURA MUNICIPAL DE SÃO LUÍS DO PARAITINGA", "ds_orgao"] = "PREFEITURA MUNICIPAL DE SÃO LUÍZ DO PARAITINGA"
df.loc[df.ds_orgao == "CÂMARA MUNICIPAL DE SÃO LUÍZ DO PARAITINGA", "ds_orgao"] = "CÂMARA MUNICIPAL DE SÃO LUZ DO PARAITINGA"
df.loc[df.ds_orgao == "PREFEITURA MUNICIPAL DE SÃO LUÍZ DO PARAITINGA", "ds_orgao"] = "PREFEITURA MUNICIPAL DE SÃO LUIZ DO PARAITINGA"

# Municípios
df2.loc[df2.ds_municipio == "Biritiba-Mirim", "ds_municipio"] = "Biritiba Mirim"
df2.loc[df2.ds_municipio == "Florínea", "ds_municipio"] = "Florínia"
df2.loc[df2.ds_municipio == "Itaóca", "ds_municipio"] = "Itaoca"
df2.loc[df2.ds_municipio == "São Luis do Paraitinga", "ds_municipio"] = "São Luíz do Paraitinga"
df2.loc[df2.ds_municipio == "São Luís do Paraitinga", "ds_municipio"] = "São Luíz do Paraitinga"
df2.loc[df2.ds_municipio == "São Luíz do Paraitinga", "ds_municipio"] = "São Luiz do Paraitinga"

# Órgãos
df2.loc[df2.ds_orgao == "CÂMARA MUNICIPAL DE BIRITIBA-MIRIM", "ds_orgao"] = "CÂMARA MUNICIPAL DE BIRITIBA MIRIM"
df2.loc[df2.ds_orgao == "PREFEITURA MUNICIPAL DE BIRITIBA-MIRIM", "ds_orgao"] = "PREFEITURA MUNICIPAL DE BIRITIBA MIRIM"
df2.loc[df2.ds_orgao == "PREFEITURA MUNICIPAL DE FLORÍNEA", "ds_orgao"] = "PREFEITURA MUNICIPAL DE FLORÍNIA"
df2.loc[df2.ds_orgao == "CÂMARA MUNICIPAL DE FLORÍNIA", "ds_orgao"] = "CÂMARA MUNICIPAL DE FLORÍNIA"
df2.loc[df2.ds_orgao == "CÂMARA MUNICIPAL DE ITAÓCA", "ds_orgao"] = "CÂMARA MUNICIPAL DE ITAOCA"
df2.loc[df2.ds_orgao == "PREFEITURA MUNICIPAL DE ITAÓCA", "ds_orgao"] = "PREFEITURA MUNICIPAL DE ITAOCA"
df2.loc[df2.ds_orgao == "CÂMARA MUNICIPAL DE SÃO LUÍS DO PARAITINGA", "ds_orgao"] = "CÂMARA MUNICIPAL DE SÃO LUÍZ DO PARAITINGA"
df2.loc[df2.ds_orgao == "PREFEITURA MUNICIPAL DE SÃO LUÍS DO PARAITINGA", "ds_orgao"] = "PREFEITURA MUNICIPAL DE SÃO LUÍZ DO PARAITINGA"
df2.loc[df2.ds_orgao == "CÂMARA MUNICIPAL DE SÃO LUÍZ DO PARAITINGA", "ds_orgao"] = "CÂMARA MUNICIPAL DE SÃO LUZ DO PARAITINGA"
df2.loc[df2.ds_orgao == "PREFEITURA MUNICIPAL DE SÃO LUÍZ DO PARAITINGA", "ds_orgao"] = "PREFEITURA MUNICIPAL DE SÃO LUIZ DO PARAITINGA"

df.to_parquet("Data/data2010to2022.parquet")
df2.to_parquet("Data/base2022subitensagrupadospormesV2.parquet")

De fato, o novo conjunto de dados gerado parece apresentar linhas a mais que o conjunto anterior. Vejamos quais são essas linhas ao remover linhas idênticas em ambos os casos (Para essa comparação, serão desconsideradas as colunas ''cd_municipio_ibge'', ''entidade_id'' e ''ds_tp_entidade'').

In [3]:
df2 = df2.filter(["ano_exercicio", "mes_referencia", "ds_municipio", "ds_orgao", "id_despesa", "vl_despesa"])

In [7]:
df_unique = (
    pd.
    merge(df, df2, indicator=True, how='outer').
    query('_merge=="left_only"').
    drop('_merge', axis=1)
)

print("Dimensão dos dados únicos: {}".format(df_unique.shape))

Dimensão dos dados únicos: (116248, 6)


In [44]:
display( df_unique.head(3) )

,ano_exercicio,mes_referencia,ds_municipio,ds_orgao,id_despesa,vl_despesa
3693,2010,1,Barretos,FUNDAÇÃO EDUCACIONAL DE BARRETOS,1101,1171022.96
3694,2010,1,Barretos,FUNDAÇÃO EDUCACIONAL DE BARRETOS,1143,1096.83
3695,2010,1,Barretos,FUNDAÇÃO EDUCACIONAL DE BARRETOS,1300,111195.51


Como podemos ver acima e nas duas linhas seguintes, o conjunto de dados utilizado anteriormente não conta com despesas referentes ao órgão ''FUNDAÇÃO EDUCACIONAL DE BARRETOS'' no ano de 2010. Entretanto, os códigos envolvidos não necessariamente foram explicitados para serem retirados. Vemos, por exemplo, que uma despesa de mesmo id (1101) está presente no conjunto ''df2'', indicando que a ausência de um órgão por completo não seria justificada pela filtragem dos códigos de despesa diretamente.

In [48]:
display( df.query("ds_orgao == 'FUNDAÇÃO EDUCACIONAL DE BARRETOS' & ano_exercicio == 2010").head(3) )

,ano_exercicio,mes_referencia,ds_municipio,ds_orgao,id_despesa,vl_despesa
3693,2010,1,Barretos,FUNDAÇÃO EDUCACIONAL DE BARRETOS,1101,1171022.96
3694,2010,1,Barretos,FUNDAÇÃO EDUCACIONAL DE BARRETOS,1143,1096.83
3695,2010,1,Barretos,FUNDAÇÃO EDUCACIONAL DE BARRETOS,1300,111195.51


In [47]:
display( df2.query("ds_orgao == 'FUNDAÇÃO EDUCACIONAL DE BARRETOS' & ano_exercicio == 2010").head(3) )

,ano_exercicio,mes_referencia,ds_municipio,ds_orgao,id_despesa,vl_despesa


In [50]:
df2.head(3)

,ano_exercicio,mes_referencia,ds_municipio,ds_orgao,id_despesa,vl_despesa
0,2010,1,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,0101,208321.50
1,2010,1,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,0301,84729.15
2,2010,1,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,1101,484631.79


Até que seja verificada esta diferença, as análises dos dados posteriores serão baseadas no conjunto de dados obtido a partir deste notebook. Dada a ínfima quantidade de dados distoantes do conjunto original ($116248/9529123 \approx 0.0122$), provavelmente este resultado não irá trazer grandes alterações para a replicação dos resultados já vistos nas análises anteriores.